In [1]:
import pandas as pd

In [2]:
df_Xtrain = pd.read_csv("Data/x_train_update.csv")

In [3]:
df_Xtrain.head()

,Unnamed: 0,designation,description,productid,imageid
0,0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046
1,1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237
2,2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978
3,3,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN,50418756,457047496
4,4,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786


In [4]:
df_Xtrain.shape

(84916, 5)

In [5]:
df_Xtrain.isna().sum()

Unnamed: 0         0
designation        0
description    29800
productid          0
imageid            0
dtype: int64

In [6]:
df_Xtest = pd.read_csv("Data/x_test_update.csv")

In [7]:
df_Xtest.head()

,Unnamed: 0,designation,description,productid,imageid
0,84916,Folkmanis Puppets - 2732 - Marionnette Et Théâ...,NaN,516376098,1019294171
1,84917,Porte Flamme Gaxix - Flamebringer Gaxix - 136/...,NaN,133389013,1274228667
2,84918,Pompe de filtration Speck Badu 95,NaN,4128438366,1295960357
3,84919,Robot de piscine électrique,<p>Ce robot de piscine d&#39;un design innovan...,3929899732,1265224052
4,84920,Hsm Destructeur Securio C16 Coupe Crois¿E: 4 X...,NaN,152993898,940543690


In [8]:
df_Xtest.shape

(13812, 5)

In [9]:
df_Xtest.isna().sum()

Unnamed: 0        0
designation       0
description    4886
productid         0
imageid           0
dtype: int64

In [10]:
df_Ytrain = pd.read_csv("Data/Y_test.csv")

In [11]:
df_Ytest = pd.read_csv("Data/Y_train.csv")

In [12]:
df_Ytrain.shape

(13812, 2)

In [13]:
df_Ytrain.head()

,Unnamed: 0,prdtypecode
0,84916,10
1,84917,10
2,84918,10
3,84919,10
4,84920,10


In [14]:
df_Ytrain.isna().sum()

Unnamed: 0     0
prdtypecode    0
dtype: int64

In [15]:
df_Ytest.shape

(84916, 2)

In [16]:
df_Ytest.head()

,Unnamed: 0,prdtypecode
0,0,10
1,1,2280
2,2,50
3,3,1280
4,4,2705


In [17]:
df_Ytest.isna().sum()

Unnamed: 0     0
prdtypecode    0
dtype: int64

In [18]:
#df_Xtrain_image_sample1 = pd.read_csv("Data/images/image_train/image_528113_product_923222.jpg")

In [19]:
df_Xtrain.columns

Index(['Unnamed: 0', 'designation', 'description', 'productid', 'imageid'], dtype='object')

In [20]:
df_Xtest.columns

Index(['Unnamed: 0', 'designation', 'description', 'productid', 'imageid'], dtype='object')

In [21]:
df_Ytrain.columns

Index(['Unnamed: 0', 'prdtypecode'], dtype='object')

In [22]:
df_Ytest.columns

Index(['Unnamed: 0', 'prdtypecode'], dtype='object')

# utilisation de EfficientNetB0 et Random Forest

In [4]:
import os
import time
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# ✅ Chargement des données
print("Chargement des données...")
df_Xtrain = pd.read_csv("img2000/X_train.csv")
df_Xtest = pd.read_csv("img2000/X_test.csv")
df_Ytrain = pd.read_csv("img2000/Y_train.csv")
df_Ytest = pd.read_csv("img2000/Y_test.csv")
print("Données chargées avec succès !")

# ✅ Traitement des valeurs manquantes
print("Traitement des valeurs manquantes...")
df_Xtrain['description'] = df_Xtrain['description'].fillna("aucune description")
df_Xtest['description'] = df_Xtest['description'].fillna("aucune description")

# ✅ Fusion des features avec les labels
print("Fusion des datasets...")
df_train = df_Xtrain.merge(df_Ytrain, on="Unnamed: 0").drop(columns=["Unnamed: 0"])
df_test = df_Xtest.merge(df_Ytest, on="Unnamed: 0").drop(columns=["Unnamed: 0"])

# ✅ Extraction des features textuelles
print("Vectorisation des descriptions textuelles...")
vectorizer = TfidfVectorizer(max_features=10000)
X_train_text = vectorizer.fit_transform(df_train["description"]).toarray()
X_test_text = vectorizer.transform(df_test["description"]).toarray()

# ✅ Chargement du modèle de feature extraction d'images
print("Chargement du modèle CNN pour l'extraction des features...")
base_model = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')
model = Model(inputs=base_model.input, outputs=base_model.output)

# ✅ Fonction d'extraction des features d'une image
def extract_image_features(image_path):
    try:
        img = image.load_img(image_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        features = model.predict(img_array)
        return features.flatten()
    except:
        return np.zeros((1280,))

# ✅ Chargement des features des images
def get_image_features(df, image_folder):
    print(f"Extraction des features pour {len(df)} images...")
    image_features = []
    zero_count = 0
    for img_id in df["imageid"]:
        img_path = os.path.join(image_folder, f"{img_id}.jpg")
        features = extract_image_features(img_path)
        if np.all(features == 0):
            zero_count += 1
        image_features.append(features)
    print(f"Nombre de vecteurs nuls : {zero_count}")
    return np.array(image_features)

# ✅ Extraction des features d'images
X_train_img = get_image_features(df_train, "img2000/image_train")
X_test_img = get_image_features(df_test, "img2000/image_test")

# ✅ Réduction de dimension des features d'images (optionnel)
print("Réduction de dimension des features d'images...")
scaler = StandardScaler()
pca = PCA(n_components=256)  # On réduit à 256 dimensions
X_train_img = pca.fit_transform(scaler.fit_transform(X_train_img))
X_test_img = pca.transform(scaler.transform(X_test_img))

# ✅ Fusion des features textuelles et visuelles
print("Fusion des features texte et image...")
X_train = np.hstack((X_train_text, X_train_img))
X_test = np.hstack((X_test_text, X_test_img))

# ✅ Définition du modèle de classification
print("Création du pipeline et début de l'entraînement...")
classifier = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)

# ✅ Entraînement du modèle
start_time = time.time()
classifier.fit(X_train, df_train["prdtypecode"])
training_time = time.time() - start_time
print(f"Modèle entraîné en {training_time:.2f} secondes.")

# ✅ Prédictions et évaluation
print("Prédiction et évaluation du modèle...")
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(df_test["prdtypecode"], y_pred)
print(f"Précision du modèle : {accuracy:.4f}")


Chargement des données...
Données chargées avec succès !
Traitement des valeurs manquantes...
Fusion des datasets...
Vectorisation des descriptions textuelles...
Chargement du modèle CNN pour l'extraction des features...
Extraction des features pour 2000 images...
Nombre de vecteurs nuls : 2000
Extraction des features pour 300 images...
Nombre de vecteurs nuls : 300
Réduction de dimension des features d'images...


C:\Users\a740407\AppData\Local\anaconda3\Lib\site-packages\sklearn\decomposition\_pca.py:685: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var


Fusion des features texte et image...
Création du pipeline et début de l'entraînement...
Modèle entraîné en 6.77 secondes.
Prédiction et évaluation du modèle...
Précision du modèle : 0.2733


# Utilisation de Restnet50 et XGBoost 

In [1]:
import os
import time
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder


# ✅ Chargement des données
print("Chargement des données...")
df_Xtrain = pd.read_csv("img2000/X_train.csv")
df_Xtest = pd.read_csv("img2000/X_test.csv")
df_Ytrain = pd.read_csv("img2000/Y_train.csv")
df_Ytest = pd.read_csv("img2000/Y_test.csv")
print("Données chargées avec succès !")

# ✅ Traitement des valeurs manquantes
print("Traitement des valeurs manquantes...")
df_Xtrain['description'] = df_Xtrain['description'].fillna("aucune description")
df_Xtest['description'] = df_Xtest['description'].fillna("aucune description")

# ✅ Fusion des features avec les labels
print("Fusion des datasets...")
df_train = df_Xtrain.merge(df_Ytrain, on="Unnamed: 0").drop(columns=["Unnamed: 0"])
df_test = df_Xtest.merge(df_Ytest, on="Unnamed: 0").drop(columns=["Unnamed: 0"])

# ✅ Extraction des features textuelles
print("Vectorisation des descriptions textuelles...")
vectorizer = TfidfVectorizer(max_features=10000)
X_train_text = vectorizer.fit_transform(df_train["description"]).toarray()
X_test_text = vectorizer.transform(df_test["description"]).toarray()

# ✅ Chargement du modèle ResNet50 pour l'extraction des features
print("Chargement du modèle CNN pour l'extraction des features...")
base_model = ResNet50(weights='resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, pooling='avg')
model = Model(inputs=base_model.input, outputs=base_model.output)

# ✅ Fonction d'extraction des features d'une image
def extract_image_features(image_path):
    try:
        img = image.load_img(image_path, target_size=(224, 224))  
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)  
        img_array = preprocess_input(img_array)  
        features = model.predict(img_array)  
        return features.flatten()
    except:
        return np.zeros((2048,))  # ResNet50 génère des features de taille 2048

# ✅ Chargement des features des images
def get_image_features(df, image_folder):
    print(f"Extraction des features pour {len(df)} images...")
    image_features = []
    for img_id in df["imageid"]:
        img_path = os.path.join(image_folder, f"{img_id}.jpg")
        image_features.append(extract_image_features(img_path))
    return np.array(image_features)

# ✅ Extraction des features d'images
X_train_img = get_image_features(df_train, "img2000/image_train")
X_test_img = get_image_features(df_test, "img2000/image_test")

# ✅ Réduction de dimension des features d'images (optionnel)
print("Réduction de dimension des features d'images...")
scaler = StandardScaler()
pca = PCA(n_components=256)  # On réduit à 256 dimensions
X_train_img = pca.fit_transform(scaler.fit_transform(X_train_img))
X_test_img = pca.transform(scaler.transform(X_test_img))

# ✅ Fusion des features texte et image
print("Fusion des features texte et image...")
X_train = np.hstack((X_train_text, X_train_img))
X_test = np.hstack((X_test_text, X_test_img))

# ✅ Définition du modèle de classification XGBoost
print("Création du modèle XGBoost et début de l'entraînement...")
classifier = XGBClassifier(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42, n_jobs=-1)

# ✅ Encodage des labels pour les rendre continus
label_encoder = LabelEncoder()
df_train["encoded_label"] = label_encoder.fit_transform(df_train["prdtypecode"])
df_test["encoded_label"] = label_encoder.transform(df_test["prdtypecode"])

# ✅ Mise à jour des labels pour XGBoost
y_train = df_train["encoded_label"]
y_test = df_test["encoded_label"]

# ✅ Entraînement du modèle
start_time = time.time()
classifier.fit(X_train, y_train)
training_time = time.time() - start_time
print(f"Modèle entraîné en {training_time:.2f} secondes.")

# ✅ Prédictions et évaluation
print("Prédiction et évaluation du modèle...")
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(df_test["prdtypecode"], y_pred)
print(f"Précision du modèle : {accuracy:.4f}")

print("Fin du processus ! 🎯")


Chargement des données...
Données chargées avec succès !
Traitement des valeurs manquantes...
Fusion des datasets...
Vectorisation des descriptions textuelles...
Chargement du modèle CNN pour l'extraction des features...
Extraction des features pour 2000 images...
Extraction des features pour 300 images...
Réduction de dimension des features d'images...


C:\Users\a740407\AppData\Local\anaconda3\Lib\site-packages\sklearn\decomposition\_pca.py:685: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var


Fusion des features texte et image...
Création du modèle XGBoost et début de l'entraînement...
Modèle entraîné en 80.91 secondes.
Prédiction et évaluation du modèle...
Précision du modèle : 0.0000
Fin du processus ! 🎯
